In [1]:
'''
This routine computes daily precipitation percentiles (80th, 90th, and 95th) using the data available in the raw folder.

Daniela Risaro
July 2025
'''

import os 
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

data_dir = "../data/raw/"
output_dir = "../data/processed/"

os.makedirs(output_dir, exist_ok=True)

## read files 
files = sorted([file for file in os.listdir(data_dir) if file.endswith(".nc") and "precipitation" in file])
years_from_files = sorted([int(file.split("_")[-1].split(".")[0]) for file in files])

print(f"\nAños disponibles: {min(years_from_files)}-{max(years_from_files)}")


base_files = [f for f in files if min(years_from_files) <= int(f.split("_")[-1].split(".")[0]) <= max(years_from_files)]
datasets = [xr.open_dataset(data_dir + f).squeeze("number", drop=True) if "number" in xr.open_dataset(data_dir + f).dims else xr.open_dataset(data_dir + f) for f in base_files]

combined = xr.concat(datasets, dim="valid_time")
combined["valid_time"] = pd.to_datetime(combined["valid_time"].values)

## compute percentiles 
combined["doy"] = combined["valid_time"].dt.dayofyear



Años disponibles: 2015-2025


In [6]:
precipitation = combined["tp"]
precipitation = precipitation.assign_coords(doy=("valid_time", combined["doy"].values))

percentiles = [80, 90, 95]
percentile_vars = {}

for p in percentiles:
    pps = (
        precipitation
        .groupby("doy")
        .reduce(np.percentile, q=p, dim="valid_time")
        .squeeze()
    )
    pps.name = f"tp_p{p}"
    pps.attrs["long_name"] = f"{p}th percentile of daily total precipitation"
    pps.attrs["units"] = "mm"
    percentile_vars[f"tp_p{p}"] = pps

tp_mean = (
    precipitation
    .groupby("doy")
    .mean(dim="valid_time")
    .squeeze()
)
tp_mean.name = "tp_mean"
tp_mean.attrs["long_name"] = "Mean daily total precipitation"
tp_mean.attrs["units"] = "mm"

percentile_vars["tp_mean"] = tp_mean

ds_out = xr.Dataset(percentile_vars)
ds_out.attrs["period"] = f"{min(years_from_files)}-{max(years_from_files)}"
ds_out.attrs["source"] = "ERA5 daily statistics (analysis), downloaded via CDS API"

out_path = os.path.join(output_dir, f"tp_mean_and_percentiles_{min(years_from_files)}_{max(years_from_files)}.nc")
ds_out.to_netcdf(out_path)